<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/introducci-n-al-Machine-Learning/blob/main/classes/class_1/class_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Chapter 5. Support Vector Machines

Una máquina de soporte vectorial (SVM) es un modelo de aprendizaje automático potente y versátil, capaz de realizar clasificación lineal o no lineal, regresión e incluso detección de valores atípicos.

Es uno de los modelos más populares en Machine Learning, y cualquier persona interesada en Machine Learning debería tenerlo en su caja de herramientas.

Las SVM son particularmente adecuadas para la clasificación de conjuntos de datos complejos de tamaño pequeño o mediano.

Este capítulo explicará los conceptos básicos de las SVM, cómo usarlas y cómo funcionan.

## Linear SVM Classification

La idea fundamental detrás de las SVM se explica mejor con algunas imágenes.

La Figura 5-1 muestra parte del conjunto de datos del iris que se presentó al final del Capítulo 4.

Las dos clases se pueden separar fácilmente con una línea recta (son linealmente separables). 

El gráfico de la izquierda muestra los límites de decisión de tres posibles clasificadores lineales.

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_1.jpg?raw=true'>

El modelo cuyo límite de decisión está representado por la línea discontinua es tan malo que ni siquiera separa las clases correctamente.

Los otros dos modelos funcionan perfectamente en este conjunto de entrenamiento, pero sus límites de decisión se acercan tanto a las instancias que estos modelos probablemente no funcionarán tan bien en nuevas instancias.

Por el contrario, la línea continua en el gráfico de la derecha representa el límite de decisión de un clasificador SVM; esta línea no solo separa las dos clases, sino que también se mantiene lo más alejada posible de las instancias de entrenamiento más cercanas.

Puede pensar en un clasificador SVM como si se ajustara a la calle más ancha posible (representada por las líneas discontinuas paralelas) entre las clases.

Esto se llama **clasificación de gran margen**.

Tenga en cuenta que agregar más instancias de capacitación "fuera de la calle" no afectará en absoluto el límite de decisión: está completamente determinado (o "respaldado") por las instancias ubicadas en el borde de la calle.

Estos casos se denominan vectores de soporte (están encerrados en un círculo en la figura 5-1).

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_2.jpg?raw=true'>

## ADVERTENCIA

Las SVM son sensibles a las escalas de características, como puede ver en la Figura 5-2: en el gráfico de la izquierda, la escala vertical es mucho más grande que la escala horizontal, por lo que la calle más ancha posible está cerca de la horizontal. 

Después de escalar características (p. ej., usando StandardScaler de Scikit-Learn), el límite de decisión en el gráfico de la derecha se ve mucho mejor.

## Soft Margin Classification
Clasificación de margen blando

Si imponemos estrictamente que todas las instancias deben estar fuera de la calle y del lado derecho, esto se denomina *clasificación de margen duro*. 

Hay dos problemas principales con la clasificación de margen duro.

Primero, solo funciona si los datos son linealmente separables. 

En segundo lugar, es sensible a los valores atípicos.

La Figura 5-3 muestra el conjunto de datos del iris con solo un valor atípico adicional: a la izquierda, es imposible encontrar un margen duro; a la derecha, el límite de decisión termina siendo muy diferente del que vimos en la Figura 5-1 sin el valor atípico, y probablemente tampoco se generalice.

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_3.jpg?raw=true'>

Para evitar estos problemas, utilice un modelo más flexible. 

El objetivo es encontrar un buen equilibrio entre mantener la calle lo más grande posible y limitar las violaciones de los márgenes (es decir, instancias que terminan en el medio de la calle o incluso en el lado equivocado).

Esto se llama *clasificación de margen suave*.

Al crear un modelo SVM con Scikit-Learn, podemos especificar una serie de hiperparámetros. 

C es uno de esos hiperparámetros.

Si lo establecemos en un valor bajo, terminamos con el modelo a la izquierda de la Figura 5-4.

Con un valor alto, obtenemos el modelo de la derecha.

Las violaciones de los márgenes son malas.

Por lo general, es mejor tener algunos de ellos.

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_4.jpg?raw=true'>

Sin embargo, en este caso, el modelo de la izquierda tiene muchas violaciones de márgenes, pero probablemente generalice mejor.

### Sugerencia

Si su modelo SVM está sobreajustado, puede intentar regularizarlo reduciendo C.

El siguiente código de Scikit-Learn carga el conjunto de datos de iris, escala las características y luego entrena un modelo SVM lineal (usando la clase LinearSVC con C=1 y la función de pérdida de bisagra, descrita en breve) para detectar flores de Iris virginica:


In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica
svm_clf = Pipeline([
("scaler", StandardScaler()),
("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

El modelo resultante se representa a la izquierda en la Figura 5-4.

Luego, como de costumbre, puede usar el modelo para hacer predicciones:

In [2]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

## NOTE

Unlike Logistic Regression classifiers, SVM classifiers do not output probabilities for each class.

Instead of using the LinearSVC class, we could use the SVC class with a linear kernel.

When creating the SVC model, we would write SVC(kernel="linear", C=1). 

Or we could use the SGDClassifier class, with SGDClassifier(loss="hinge",
alpha=1/(m*C)). 

This applies regular Stochastic Gradient Descent (see Chapter 4) to train a linear SVM classifier. 

It does not converge as fast as the LinearSVC class, but it can be useful to handle online classification tasks or huge datasets that do not fit in memory (out-of-core training).

### TIP

The LinearSVC class regularizes the bias term, so you should center the training set first by subtracting its mean. 

This is automatic if you scale the data using the StandardScaler. 

Also make sure you set the loss hyperparameter to "hinge", as it is not the default value. 

Finally, for better performance, you should set the dual hyperparameter to False, unless there are more features than training instances (we will discuss duality later in the chapter).

## Nonlinear SVM Classification

Although linear SVM classifiers are efficient and work surprisingly well in many cases, many datasets are not even close to being linearly separable. 

One approach to handling nonlinear datasets is to add more features, such as polynomial features (as you did in Chapter 4); in some cases this can result in a linearly separable dataset. 

Consider the left plot in Figure 5-5: it represents a simple dataset with just one feature, $x_{1}$.

This dataset is not linearly separable, as you can see. 

But if you add a second feature $x_{2} = (x_{1})^{2}$ , the resulting 2D dataset is perfectly linearly separable.